In [ ]:
import pandas as pd
import re
pd.set_option('display.max_columns', None)


In [ ]:

matrix_df=pd.read_csv('/home/bram/Project_PD/PD project/Feature matrix/02_ICD/matrix_incl_icd.csv')
print(len(matrix_df))

In [ ]:
print(matrix_df.head())

In [ ]:
meds_df=pd.read_csv('/home/bram/Project_PD/PD project/Feature matrix/03_MEDFEAT/allmedication_new.csv')
print(len(meds_df))
print(meds_df.head())
meds_df['medDate']=meds_df['CreateDate']


In [ ]:
meds_df=meds_df.drop('CreateDate', axis=1)

In [ ]:
print(meds_df.head())
print(len(meds_df))

In [ ]:
matrix_df['CreateDate']= pd.to_datetime(matrix_df['CreateDate'])
meds_df['medDate'] = pd.to_datetime(meds_df['medDate'])
# ICDs_STF['ICD Date'] = pd.to_datetime(ICDs_STF['CreateDate'])


In [ ]:


# # Keep the original data in a separate column
# meds_data['original_medDate'] = meds_data['medDate']
# # Convert to datetime with errors='coerce'
# meds_data['medDate'] = pd.to_datetime(meds_data['medDate'], errors='coerce')

# # Filter rows where 'medDate' is NaT
# coerced_entries = meds_data[meds_data['medDate'].isna()]

# print("Coerced Entries:")
# print(coerced_entries)

In [ ]:

# # Sample DataFrame for existing matrix with note dates and patient IDs
# matrix_data = {
#     'BDSPPatientID': [1, 2, 3, 4, 5, 6],
#     'CreateDate': pd.to_datetime(['2021-01-15', '2021-05-20', '2021-03-10', '2021-08-25', '2021-11-05', '2021-11-05'])
# }

# matrix_df = pd.DataFrame(matrix_data)

# # Sample DataFrame for medications with medication dates and patient IDs
# meds_data = {
#     'BDSPPatientID': [ 2, 2, 3, 3, 5, 5, 1],
#     'medicationdescription': [
#         'Levodopa', 'Dopamineagonists', 'Apomorphine', 'Rotigotine', 'Sinemet', 'Carbidopa', 'aaRopinirole'
#     ],
#     'medDate': pd.to_datetime(['2021-12-01', '2021-06-01', '2021-02-15', '2021-09-15', '2021-11-20', '2021-10-01', '2021-5-01'])
# }

# meds_df = pd.DataFrame(meds_data)

# Merge the DataFrames on BDSPPatientID
merged_df = pd.merge(matrix_df, meds_df, on='BDSPPatientID', how='left')

# Check for date timeline
merged_df['MedicationWithin6Months'] = merged_df.apply(
    lambda row: (row['CreateDate'] >= row['medDate'] - pd.DateOffset(months=6)) and 
                (row['CreateDate'] <= row['medDate'] + pd.DateOffset(months=6)) if pd.notnull(row['medDate']) else False, axis=1)

# Filter merged_df to include only rows where MedicationWithin6Months is True
filtered_merged_df = merged_df[merged_df['MedicationWithin6Months']].copy()

print('\n filteMedicationWithin6Months column', merged_df['MedicationWithin6Months'])
# Aggregate medication information per patient
aggregated_meds = filtered_merged_df.groupby('BDSPPatientID')['medication'].apply(lambda x: ' '.join(x)).reset_index()

# Merge aggregated medication information back with the matrix DataFrame
matrix_merged_df = pd.merge(matrix_df, aggregated_meds, on='BDSPPatientID', how='left')

# Generate regex patterns for the feature matrix
def generate_patterns(medications):
    patterns = {med: re.compile(rf"{med}", re.IGNORECASE) for med in medications}
    return patterns

medication_patterns = generate_patterns([
    'Levodopa', 'Carbidopa', 'Dopamine', 'Dopamineagonists', 'Monoamineoxidase', 'Catechol-O-methyltransferase', 
    'Apomorphine', 'Rotigotine', 'Pramipexole', 'Ropinirole', 'Selegiline', 'Rasagiline', 'Entacapone', 
    'Tolcapone', 'Trihexyphenidyl', 'Benztropine', 'Amantadine', 'Stalevo', 'Neupro', 'Mirapex', 'Requip', 
    'Sinemet', 'Madopar', 'Comtan', 'Azilect', 'Eldepryl', 'Zelapar', 'Bromocriptine', 'Pergolide', 
    'Safinamide', 'Opicapone', 'Zonisamide', 'Duopa', 'Inbrija', 'Xadago', 'Nuplazid', 'Ongentys', 'Nourianz', 
    'RequipXL', 'Apokyn', 'Zydis', 'Tasmar', 'COMT', 'MAO', 'Istradefylline', 'Rytary'
])

# Function to check for the presence of medication patterns
def check_medications(text, patterns):
    matches = {med: 0 for med in patterns.keys()}  # Initialize matches dictionary with 0
    if pd.notnull(text):
        for med, pattern in patterns.items():
            if pattern.search(text):
                matches[med] = 1
    return matches

# Apply the pattern checking to the aggregated medication column
matrix_merged_df['medication_features'] = matrix_merged_df['medication'].apply(lambda desc: check_medications(desc, medication_patterns) if pd.notnull(desc) else {med: 0 for med in medication_patterns.keys()})

# Convert the medication features to a DataFrame
medication_features_df = pd.DataFrame(matrix_merged_df['medication_features'].tolist())

# Concatenate with the original matrix DataFrame, ensuring all rows are included
result_df = pd.concat([matrix_df, medication_features_df], axis=1)

# Remove any extra rows that were added during the merge
result_df = result_df.dropna(subset=['BDSPPatientID']).reset_index(drop=True)

# Display the result
print(result_df)


In [ ]:
sum_med=result_df['Istradefylline'].sum()
print(sum_med)

In [ ]:
# pseudo

result_df.to_csv('finalmatrix.csv', index=False)